In [1]:
import pandas as pd
import numpy as np
import requests as re
from tqdm import *
from ratelimiter import RateLimiter
from bs4 import BeautifulSoup

In [14]:
url = 'https://www.indeed.com/r/Vidya-T/a834f22bf29502a2'
this_content = re.get(url)

In [15]:
urls_splits =url.split('/')
urls_splits[len(urls_splits)-1]

'a834f22bf29502a2'

In [4]:
soup = BeautifulSoup(this_content.content,'html.parser')

In [5]:
a=soup.find('p', attrs={'id':'headline_location'})
a.contents[0]

AttributeError: 'NoneType' object has no attribute 'contents'

In [51]:

@RateLimiter(max_calls=1, period=4)
def get_resume(link, rb):
    
    this_content = re.get(url)
    soup = BeautifulSoup(this_content.content,'html.parser')
    
    mydict= {}
    mydict['rb']=rb
    urls_splits =url.split('/')
    mydict['id'] = urls_splits[len(urls_splits)-1]
    mydict['summary_title'] = soup.find('h1', attrs={'class': 'fn'}).contents[0]
    mydict['location']  = soup.find('p', attrs={'id':'headline_location'}).contents[0]
    work_ex= soup.find('div',attrs={'id': 'work-experience-items'})
    if len(work_ex.contents)>0:
        mydict['current_job_company']= work_ex.contents[0].find('div', attrs= {'class':'work_company'}).contents[0].contents
        mydict['current_job_title']= work_ex.contents[0].find('p', attrs={'class':'work_title title'}).contents[0]
        mydict['current_job_desc'] = work_ex.contents[0].find('p', attrs={'class':'work_description'}).contents
        mydict['current_job_duration'] = work_ex.contents[0].find('p', attrs={'class':'work_dates'}).contents[0]

    if len(work_ex.contents)>1:
        # Previous experience is available
        mydict['prev_job_company']= work_ex.contents[1].find('div', attrs= {'class':'work_company'}).contents[0].contents
        mydict['prev_job_title']= work_ex.contents[1].find('p', attrs={'class':'work_title title'}).contents[0]
        mydict['prev_job_desc'] = work_ex.contents[1].find('p', attrs={'class':'work_description'}).contents
        mydict['prev_job_duration'] = work_ex.contents[1].find('p', attrs={'class':'work_dates'}).contents[0]

    work_duration_list=[]
    # Get total work exp:
    if len(work_ex.contents)>0:
        for work in work_ex.contents:
            this_duration = work.find('p', attrs={'class':'work_dates'}).contents[0]
            work_duration_list.append(this_duration)

    mydict['total_exp_durations'] = work_duration_list

    educations = soup.find('div',attrs={'id': 'education-items'})
    if len(educations.contents)>0:
        mydict['education1_title']=educations.contents[0].find('p', attrs={'class':'edu_title'}).contents[0]
        mydict['education1_college']=educations.contents[0].find('div', attrs={'class':'edu_school'}).contents[0].contents[0]
        mydict['education1_duration']=educations.contents[0].find('p', attrs={'class':'edu_dates'}).contents[0]

    if len(educations.contents)>1:
        mydict['education2_title']=educations.contents[1].find('p', attrs={'class':'edu_title'}).contents[0]
        mydict['education2_college']=educations.contents[1].find('div', attrs={'class':'edu_school'}).contents[0].contents[0]
        mydict['education2_duration']=educations.contents[1].find('p', attrs={'class':'edu_dates'}).contents[0]

    if len(educations.contents)>2:
        mydict['education3_title']=educations.contents[2].find('p', attrs={'class':'edu_title'}).contents[0]
        mydict['education3_college']=educations.contents[2].find('div', attrs={'class':'edu_school'}).contents[0].contents[0]
        mydict['education3_duration']=educations.contents[2].find('p', attrs={'class':'edu_dates'}).contents[0]



    # Skill
    skill_list=[]
    skills = soup.find('div',attrs={'id': 'skills-items'})
    if skills is not None:
        skill_set = skills.contents[0].contents[0].contents
        if len(skill_set)>0:
            for skill in skill_set:
                try:
                    skill_list.append(skill.contents)
                except:
                    pass
    mydict['skils']=skill_list

    # links
    links_list=[]
    links= soup.find('div',attrs={'id':'link-items'})
    if links is not None:
        links_set =links.contents
        for link in links_set:
            links_list.append(link.contents[0].contents[0].contents[0].contents[0])

    mydict['links']=links_list


    # additional_info
    add_info = soup.find('div', attrs={'id':'additionalinfo-section'})
    if add_info is not None:
        mydict['additional_info']=add_info.contents[0].contents[0].contents


    certi_list=[]
    certifications = soup.find('div', attrs={'id':'certification-items'})
    if certifications is not None:
        for certi in certifications.contents:
            certi_list.append(certi.contents[0].contents[0].contents[0])
    mydict['certifications']=certi_list

    award_list=[]
    awards=soup.find('div', attrs={'id':'award-items'})
    if awards is not None:
        for one_award in awards:
            award_list.append(one_award.contents[0].contents[0].contents[0])

    mydict['awards'] = award_list
    return mydict


In [10]:
pd.DataFrame(data=[mydict])

,additional_info,awards,certifications,current_job_company,current_job_desc,current_job_duration,current_job_title,education1_college,education1_duration,education1_title,...,education2_title,id,links,prev_job_company,prev_job_desc,prev_job_duration,prev_job_title,skils,summary_title,total_exp_durations
0,"[SKILLS , <br/>, • Data Analytics Tools/Lan...",[],[],[Booxby],[• Booxby is a machine learning platform so...,December 2017 to Present,Data Scientist Intern,"University of Illinois at Urbana-Champaign, Ch...",August 2014 to August 2015,MS Business in Data Science,...,BA in Economics,404950f566637da6,[http://github.com/kellypeng],[Gausscode Technology Inc],[• Worked as the company's first data analy...,February 2016 to May 2017,Data Analyst,[[Amazon Elastic Compute Cloud (Less than 1 ye...,Data Scientist Intern,"[December 2017 to Present, February 2016 to Ma..."


In [20]:
all_links = pd.read_csv('links3.csv')

In [21]:
all_links.head(), all_links.shape

(   Unnamed: 0                                              links          rb
 0           0  https://www.indeed.com/r/Tracy-Ruan/c47f7ac095...  6-10 years
 1           1  https://www.indeed.com/r/Rebecca-Mashaido/b3fa...  6-10 years
 2           2  https://www.indeed.com/r/Zhisheng-Lin/e307d3f4...  6-10 years
 3           3  https://www.indeed.com/r/Tae-Hyon-Lee/f09d6780...  6-10 years
 4           4  https://www.indeed.com/r/Wendy-Gao/1828f774d1c...  6-10 years,
 (11213, 3))

In [52]:
all_resumes_parsed =[]
for i, row in tqdm(all_links.iterrows()):
    try:
        with open('test.csv', 'a+') as f: 
            this_parsed = get_resume(row['links'], row['rb'])
            if i==0:
                w.writeheader()
            w = csv.DictWriter(f, this_parsed.keys())
            w.writerow(this_parsed)
    except:
        print(row['links'])
        raise
    all_resumes_parsed.append(this_parsed)

0it [00:00, ?it/s]

https://www.indeed.com/r/Tracy-Ruan/c47f7ac095973653?sp=0


AttributeError: 'NoneType' object has no attribute 'contents'

In [23]:
from ratelimiter import RateLimiter

In [46]:
all_links.loc[409]['links']

'https://www.indeed.com/r/Sherry-Kawing-Lau/d46d95cbdbacb8b3?sp=0'

In [17]:
all_resumes_parsed[len(all_resumes_parsed)-1]

NameError: name 'all_resumes_parsed' is not defined

In [49]:
import csv

my_dict = {"test": 4, "testing": 5}

with open('test.csv', 'a+') as f:  # Just use 'w' mode in 3.x
    w = csv.DictWriter(f, my_dict.keys())
    #w.writeheader()
    w.writerow(my_dict)